In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [ ]:
"""Task 1:
given a list of NGS sequences, find out which sequences are relative abundance in the dataset.
"""

In [ ]:
"""Task 2:
given a list of NGS sequences, find out the Mass V.S. Shift map
"""

In [ ]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx'
df_mock = load_data(path)
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S6to10combined_20ul_40min.xlsx'
df_rsv = load_data(path)
df_mock.shape, df_rsv.shape

In [ ]:
path = '/Users/xyuan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/UTMB1.xlsx'
df_intact = load_data(path)
df_intact.shape

In [ ]:
# remove adducts from df_glu
adducts = [K, Na]
dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_mock, df_mock, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_mock_without_adducts = df_mock.drop(dfm_adducts.index)
df_mock_adducts_only = dfm_adducts.copy()
print(df_mock.shape[0], df_mock_without_adducts.shape[0])

dfm_adducts_list = list()
for adduct in adducts:
    dfm_r = match_dfs(df_rsv, df_rsv, shift=adduct)
    dfm_adducts_list.append(dfm_r)
dfm_adducts = pd.concat(dfm_adducts_list).drop_duplicates()
df_rsv_without_adducts = df_rsv.drop(dfm_adducts.index)
df_rsv_adducts_only = dfm_adducts.copy()
print(df_rsv.shape[0], df_rsv_without_adducts.shape[0])

In [ ]:
df_mock_sample = df_mock.copy()
df_rsv_sample = df_rsv.copy()
df_mock_sample.shape, df_rsv_sample.shape

In [ ]:
df_mock_adducts, df_rsv_adducts = df_mock_adducts_only, df_rsv_adducts_only
df_mock_adducts.shape, df_rsv_adducts.shape

Task 1

In [ ]:
df = pd.read_excel('~/Downloads/a.xlsx')
df.shape

In [ ]:
shifts = [0, 14]

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
    bcr = base_calling_random(df, base_only=True)
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

def shift_hits(df_ngs, df_src, shifts):
    PROCESSES = 8
    params = [(df_ngs, df_src, shift) for shift in shifts]
    with multiprocessing.Pool(PROCESSES) as pool:
        df_basecallings = pool.starmap(func, params)

    df_hits = pd.concat(df_basecallings).drop_duplicates()
    return df_hits


In [ ]:
df_ngs_src = pd.read_excel('~/Downloads/a.xlsx')
# df_ngs_src = pd.read_excel('~/Downloads/EJ_mock.xlsx')
# df_ngs_src = pd.read_excel('~/Downloads/EJ_rsv.xlsx')
# df_ngs_src = pd.read_excel('~/Downloads/EJ_summary.xlsx')

# ngs_path = '~/Documents/BioPharmaFinder/BaoLab/NGS/tRNA-Glu-CTC_CNvRSV_EJ.xlsx'
# df_ngs_src = pd.read_excel(ngs_path, 1)
df_ngs_src.shape, df_ngs_src.head()

In [ ]:
def T2U(seq):
    """Change input sequence from DNA to RNA stype, 'T' to 'U' 
    """
    # print('input', seq)
    seq = seq.replace('T', 'U').replace('t', 'U')
    return seq

df_ngs_src['seq'] = df_ngs_src.seqs.apply(T2U)
df_ngs_src.head()

In [ ]:
df_ngs_src = df_ngs_src.drop_duplicates(subset=['seqs'])
df_ngs_src.shape[0]

In [ ]:
dft = df_ngs_src.copy()
# df_sample = df_mock_sample
df_sample = df_rsv_sample
for idx, row in dft.iterrows():
#     if len(row['seqs']) < 65:
#         continue
    seq = row['seq']
    df_5p = pd.read_json(row['MassLadder5p'])
    df_3p = pd.read_json(row['MassLadder3p'])
    
    df_5p['Base'] = list(seq)
    df_3p['Base'] = list(seq[::-1])
    
    df_5p['Idx'] = np.array(df_5p.index)+1
    df_3p['Idx'] = 75-np.array(df_3p.index)
    
    df_hits_5p = shift_hits(df_5p, df_sample, shifts)
    df_hits_3p = shift_hits(df_3p, df_sample, shifts)
    print(idx, seq, '5´ hits:', df_hits_5p.shape[0], '3´ hits:', df_hits_3p.shape[0])
    df_ngs_src.loc[idx, 'Hit5Ladder'] = df_hits_5p.shape[0]
    df_ngs_src.loc[idx, 'Hit3Ladder'] = df_hits_3p.shape[0]

# df_ngs_src.to_excel('~/Downloads/summary_result.xlsx')
df_ngs_src.to_excel('~/Downloads/a_result.xlsx')
    
#     break

In [ ]:
df_hits = shift_hits(df_5p, df_mock_sample, shifts)
df_hits.shape

In [ ]:
plotly_zone(df_hits, y='Shift')
df_hits

In [ ]:
df.head()

In [ ]:
# p-seq-OH, OH-seq-p, mass sum + h2o

# p-seq-p, mass sum + h2o + 80

# OH-seq-OH, mass sum + h2o - 80

In [ ]:
mass_glu = 147.1

In [ ]:
def reg_mass(seq):
    """given a sequence(contains ACGU only), calculate the intact mass
    """
    mass = 0
    bases = {'A': 329.0525,
            'C': 305.0413,
            'G': 345.0474,
            'U': 306.0253}
    
    for base in bases:
        num = seq.count(base)
        mass += num * bases.get(base)

    mass += H2O
    return mass


In [ ]:
# mass = reg_mass(seq)
# mass_p = mass + 80
# mass_oh = mass - 80
# mass_amino = mass + mass_glu - H2O

# mass, mass_p, mass_oh, mass_amino

In [ ]:
dft = df_ngs_src.dropna(subset=['seq']).copy()
df_sample = df_rsv_sample
for idx, row in dft.iterrows():
    seq = row['seq']
#     print(idx, row['seqs'])
    mass = reg_mass(seq)
    df_ngs_src.loc[idx, 'MassReg'] = mass + C+C+A if len(seq)>=72 else 0
    df_ngs_src.loc[idx, 'MassP'] = mass + 80
    df_ngs_src.loc[idx, 'MassOH'] = mass - 80
    df_ngs_src.loc[idx, 'MassAmino'] = mass + mass_glu - H2O

In [ ]:
df_ngs_src.head()

In [ ]:
# df = df_ngs_src.copy()
# df.rename(columns={'MassP': "Mass"}, inplace=True)
# plotly_zone(df, x='Mass', y='Vol')
plotly_zones(df, dfm_seq, y='Vol')
# df = df_ngs_src[['seq', 'MassH']].copy()
# df.rename(columns={'MassOH': "Mass"}, inplace=True)
# df = df_ngs_src[['seq', 'MassAmino']].copy()
# df.rename(columns={'MassAmino': "Mass"}, inplace=True)

In [ ]:
df[['name', 'b', 'seqs', 'Vol', 'Mass']]

In [ ]:
dfm_seq, dfm = peer_dfs(df, df_intact)
dfm_seq.shape, dfm.shape

In [ ]:
dfm_seq#[['seq', 'Mass', 'Vol']]#.iloc[:10]
# dfm_seq[['seq', 'Mass', 'Vol']].sort_values('Mass').groupby(['Mass']).head(1)

In [ ]:
# shifts
# dft.sort_values('Mass').Mass
plotly_zone(df_mock_sample)

In [ ]:
dft = dfm_seq.copy()
df_sample = df_mock_sample
shifts = [0, 14]
# masses = [11032.3456, 11644.3962]
masses = [12582.5473]
# df_sample = df_rsv_sample
for idx, row in dft.iterrows():
    seq = row['seq']
    df_5p = pd.read_json(row['MassLadder5p'])
    df_5p['Base'] = list(seq)
    df_5p['Idx'] = np.array(df_5p.index)+1
    
    df_hits_5p = shift_hits(df_5p, df_sample, shifts)
    dfm_seq.loc[idx, 'Hit5Count'] = df_hits_5p.shape[0]
    dfm_seq.loc[idx, 'Hit5Ladder'] = df_hits_5p.to_json()

dfm_seq.to_excel('~/Downloads/b_result.xlsx')

In [ ]:
dfm_seq.sort_values('Hit5Count', ascending=False)[['seq', 'Mass', 'Hit5Count']].iloc[:30]

In [ ]:
dft = dfm_seq.copy()
df_sample = df_mock_sample
shifts = [0, 14, 16, 18]
# masses = [11032.3456, 11644.3962]
masses = [12582.5473]
# df_sample = df_rsv_sample
for idx, row in dft.iterrows():
    if not abs(row['Mass'] - masses[0]) < 1E-3:
#     if not (abs(row['Mass'] - masses[0]) < 1E-3 or abs(row['Mass'] - masses[1]) < 1E-3):
#         print(row['Mass'])
        continue
#     if len(row['seqs']) < 65:
#         continue
    seq = row['seq']
    df_5p = pd.read_json(row['MassLadder5p'])
#     df_3p = pd.read_json(row['MassLadder3p'])
    
    df_5p['Base'] = list(seq)
#     df_3p['Base'] = list(seq[::-1])
    
    df_5p['Idx'] = np.array(df_5p.index)+1
#     df_3p['Idx'] = 75-np.array(df_3p.index)
    
    df_hits_5p = shift_hits(df_5p, df_sample, shifts)
#     df_hits_3p = shift_hits(df_3p, df_sample, shifts)
    print(idx, seq, row['Mass'], '5´ hits count:', df_hits_5p.shape[0], '5´ hits:\n', df_hits_5p.sort_values(['Shift', 'Mass']))
    plotly_zone(df_hits_5p, y='Shift')

In [ ]:
plotly_zones(df_intact, dfm, y='RT')
# plotly_zone(dfm_mock)
print(df_intact[(df_intact.Mass>10656)&(df_intact.Mass<14533)].sort_values('Mass'))
print(dfm.sort_values('Mass'))

In [ ]:
dfm_seq.to_excel('~/Downloads/tmp.xlsx')

In [ ]:
target = 11032.2456
dfm_seq[(dfm_seq.Mass>target-0.5)&(dfm_seq.Mass<target+0.5)]

In [ ]:
bcr = base_calling_random(dfm)

In [ ]:
plotly_basecalling(dfm, bcr[1])

In [ ]:
dfm_seq.seq.astype(str)
# dfm_seq.drop_duplicates(subset=[seq])

In [ ]:
dfm_seq.info()